# 2019 

## Day 1

In [ ]:
input1 = Input(1, line_parser=int, file_template='data/advent2019/input{}.txt')

fuel_needed = lambda mass: mass // 3 - 2

assert 2 == quantify([12], fuel_needed)
assert 2 == quantify([14], fuel_needed)
assert 654 == quantify([1969], fuel_needed)
assert 33583 == quantify([100756], fuel_needed)

assert 3311492 == quantify(input1, fuel_needed)

In [ ]:
def part_2(mass):
    total_fuel = 0
    while True:
        mass = fuel_needed(mass)
        if mass < 0: return total_fuel
        total_fuel += mass

assert 966 == part_2(1969)
assert 50346 == part_2(100756)
assert 4964376 == quantify(input1, part_2)

## Day 2

In [ ]:
def part_1(code, state=None):
    c = deepcopy(code)
    if state: c[1:3] = state
    try:
        for (op, x, y, out) in chunks(c, 4):
            if op == 1: c[out] = c[x] + c[y]
            if op == 2: c[out] = c[x] * c[y]
    except ValueError:
        return c

assert [2,0,0,0,99] == part_1([1,0,0,0,99])
assert [2,3,0,6,99] == part_1([2,3,0,3,99])
assert [30,1,1,4,2,5,6,0,99] == part_1([1,1,1,4,99,5,6,0,99])

to_list = lambda text: list(map(int, re.findall(r'-?\d+', text)))
input2, = Input(2, line_parser=to_list, file_template='data/advent2019/input{}.txt')

assert 3409710 == part_1(input2, [12, 2])[0]

In [ ]:
def part_2(code):
    for noun, verb in product(rangei(0, 99), repeat=2):
        if 19690720 == part_1(code, (noun, verb))[0]:
            return 100 * noun + verb
        
assert 7912 == part_2(input2)

## Day 3

In [ ]:
headings = {'R': right, 'L': left, 'U': up, 'D': down}

def part_1(p0, p1):
    return min(manhattan(0, z) for z in set(p0) & set(p1))

def parse(path):
    cur, points = 0, []
    for (heading, distance) in re.findall("(\w)(\d*)", path):
        for _ in range(int(distance)):
            cur += headings[heading]
            points.append(cur)
    return points
    
test_1 = "R75,D30,R83,U83,L12,D49,R71,U7,L72\nU62,R66,U55,R34,D71,R55,D58,R83"
test_1 = mapt(parse, test_1.split())

test_2 = "R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51\nU98,R91,D20,R16,D67,R40,U7,R15,U6,R7"
test_2 = mapt(parse, test_2.split())

assert 159 == part_1(*test_1)
assert 135 == part_1(*test_2)

input3 = Input(3, line_parser=parse, file_template='data/advent2019/input{}.txt')
assert 399 == part_1(*input3)

In [ ]:
def part_2(p0, p1):
    return min(p0.index(i) + p1.index(i) for i in set(p0) & set(p1)) + 2
    
assert 610 == part_2(*test_1)
assert 410 == part_2(*test_2)

assert 15678 == part_2(*input3)

## Day 4

In [ ]:
def is_valid_password(s):
    return any(a == b for a, b in zip(s[:-1], s[1:])) and \
           all(a <= b for a, b in zip(s[:-1], s[1:]))

assert True == is_valid_password('111111')
assert False == is_valid_password('223450')
assert False == is_valid_password('123789')

input4 = mapt(str, rangei(152085, 670283))

assert 1764 == quantify(input4, is_valid_password)

In [ ]:
def is_valid_password_2(s):
    return is_valid_password(s) and \
           any(sum(1 for _ in k) == 2 for _, k in groupby(s))

assert True == is_valid_password_2('112233')
assert False == is_valid_password_2('123444')
assert True == is_valid_password_2('111122')

assert 1196 == quantify(input4, is_valid_password_2)  # 802 is too low

## Day 5

In [ ]:
def execute(intcode, my_input=1, my_output=print):
    intcode = deepcopy(intcode)
    num_operands = (0, 3, 3, 1, 1, 2, 2, 3, 3)
    ip = 0
    while intcode[ip] != 99:
        modes = [int(x) for x in f"{intcode[ip]:0>5}"[:3]][::-1]
        instruction = int(f"{intcode[ip]:0>5}"[3:])
        operands = [intcode[ip + x + 1] if modes[x] else intcode[intcode[ip + x + 1]] for x in
                    range(num_operands[instruction])]
        if instruction == 1:
            intcode[intcode[ip + 3]] = operands[0] + operands[1]
        elif instruction == 2:
            intcode[intcode[ip + 3]] = operands[0] * operands[1]
        elif instruction == 3:
            intcode[intcode[ip + 1]] = my_input
        elif instruction == 4:
            my_output(operands[0])
        elif instruction == 5:
            ip = operands[1] - 3 if operands[0] != 0 else ip
        elif instruction == 6:
            ip = operands[1] - 3 if operands[0] == 0 else ip
        elif instruction == 7:
            intcode[intcode[ip + 3]] = int(operands[0] < operands[1])
        elif instruction == 8:
            intcode[intcode[ip + 3]] = int(operands[0] == operands[1])
        else:
            assert False
        ip += num_operands[instruction] + 1


input5, = Input(5, line_parser=comma_integers, file_template='data/advent2019/input{}.txt')

execute(input5)

In [ ]:
execute(input5, my_input=5)

## Day 6

In [ ]:
def generate_orbit_map(data):
    return dict(row.split(')')[::-1] for row in data)


def trace_path(orbit_map, orbit):
    path = set()
    while orbit != 'COM':
        path.add(orbit := orbit_map[orbit])
    return path


orbit_map = generate_orbit_map(Input(6, file_template='data/advent2019/input{}.txt'))
assert 144909 == sum(len(trace_path(orbit_map, planet)) for planet in orbit_map)

In [ ]:
assert 259 == len(trace_path(orbit_map, 'YOU') ^ trace_path(orbit_map, 'SAN'))

## Day 7

In [ ]:
def execute(code, input_queue):
    code = deepcopy(code)
    num_operands = (0, 3, 3, 1, 1, 2, 2, 3, 3)
    ip = 0
    while code[ip] != 99:
        modes = [int(x) for x in f"{code[ip]:0>5}"[:3]][::-1]
        instruction = int(f"{code[ip]:0>5}"[3:])
        operands = [code[ip + x + 1] if modes[x] else code[code[ip + x + 1]] for x in range(num_operands[instruction])]
        # print(ip, instruction, modes, operands, code)
        if instruction == 1:
            code[code[ip + 3]] = operands[0] + operands[1]
        elif instruction == 2:
            code[code[ip + 3]] = operands[0] * operands[1]
        elif instruction == 3:
            code[code[ip + 1]] = input_queue.pop()
        elif instruction == 4:
            yield operands[0]
        elif instruction == 5:
            ip = (operands[1] - 3) if operands[0] != 0 else ip
        elif instruction == 6:
            ip = (operands[1] - 3) if operands[0] == 0 else ip
        elif instruction == 7:
            code[code[ip + 3]] = int(operands[0] < operands[1])
        elif instruction == 8:
            code[code[ip + 3]] = int(operands[0] == operands[1])

        ip += num_operands[instruction] + 1
    return StopIteration


def parse_integers(day, file_template='data/advent2019/input{}.txt'):
    return list(int(i) for line in open(file_template.format(day)) for i in line.split(','))


input5 = parse_integers(5)
assert 3176266 == next(execute(deepcopy(input5), input_queue=deque((5,))))


def max_thruster_signal(code):
    max_signal = -infinity
    for p in permutations(range(5)):
        y = 0
        for i in range(5):
            y = next(execute(code[:], input_queue=deque((y, p[i]))))
        max_signal = max(y, max_signal)
    return max_signal


assert 999 == next(execute([3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99], input_queue=deque((7,))))
assert 1000 == next(execute([3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99], input_queue=deque((8,))))
assert 1001 == next(execute([3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99], input_queue=deque((88,))))

assert 43210 == max_thruster_signal([3, 15, 3, 16, 1002, 16, 10, 16, 1, 16, 15, 15, 4, 15, 99, 0, 0])
assert 54321 == max_thruster_signal([3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0])
assert 65210 == max_thruster_signal([3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0])

input7 = parse_integers(7)
assert 262086 == max_thruster_signal(input7)

In [ ]:
def max_feedback_thruster_signal(code):
    max_signal = -infinity
    for phases in permutations(range(5, 10)):
        input_queues = [deque((p,)) for p in phases]
        thrusters = [execute(code, d) for d in input_queues]
        y, is_halted = 0, False
        while not is_halted:
            try:
                for (input_queue, thruster) in zip(input_queues, thrusters):
                    input_queue.appendleft(y)
                    y = next(thruster)
            except StopIteration:
                max_signal = max(y, max_signal)
                is_halted = True
    return max_signal

assert 139629729 == max_feedback_thruster_signal( [3, 26, 1001, 26, -4, 26, 3, 27, 1002, 27, 2, 27, 1, 27, 26, 27, 4, 27, 1001, 28, -1, 28, 1005, 28, 6, 99, 0, 0, 5])
assert 18216 == max_feedback_thruster_signal([3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54, -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4, 53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10])

assert 5371621 == max_feedback_thruster_signal(input7)

## Day 8

In [ ]:
input8 = wrap(cat(Input(8, file_template='data/advent2019/input{}.txt')), 25 * 6)
fewest_zeros = min(input8, key=lambda layer: layer.count('0'))
assert 2159 == fewest_zeros.count('1') * fewest_zeros.count('2')

In [ ]:
def parse_pixel(*layers):
    return next(dropwhile('2'.__eq__, layers))


parsed_pixles = ''.join(map(parse_pixel, *input8))
print('\n'.join(l.replace('0', ' ') for l in chunks(cat(parsed_pixles), 25)))

## Day 10

In [ ]:
def parse(lines):
    return set(complex(j, i) for (i, col) in enumerate(lines) for (j, num) in enumerate(col) if num == '#')


def max_visible(asteroids):
    return max(((a, len(set(phase(b-a) for b in asteroids - {a}))) for a in asteroids), key=itemgetter(1))



small = '.#..#\n.....\n#####\n....#\n...##'.split('\n')
assert (3+4j, 8) == max_visible(parse(small))


assert (5+8j, 33) == max_visible(parse('......#.#.\n#..#.#....\n..#######.\n.#.#.###..\n.#..#.....\n..#....#.#\n#..#....#.\n.##.#..###\n##...#..#.\n.#....####'.split('\n')))

test = '''.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##'''.split('\n')

assert (11+13j, 210) == max_visible(parse(test))

input10 = parse(Input(10, file_template='data/advent2019/input{}.txt'))
assert ((23+29j), 263) == max_visible(input10)

In [ ]:
def unwrap(phase):
    return phase + (0 if phase >= 0 else 2 * pi)


def closest_circle(asteroids, a=23 + 29j):
    # set of all asteroids with minimum len for each phase
    all_asteroids = sorted(set(((b, unwrap(phase(1j * (b - a))), abs(b - a)) for b in asteroids - {a})), key=itemgetter(1))
    return (min(g, key=itemgetter(2))[0] for (_, g) in groupby(all_asteroids, key=itemgetter(1)))


def laser(asteroids, base=23 + 29j, stop_at=200):
    asteroids = asteroids.copy()
    i, a = 0, None
    while True:
        c = closest_circle(asteroids, base)
        for a in c:
            asteroids -= {a}
            i += 1
            if i == stop_at:
                return a


assert (8 + 2j) == laser(parse(test), 11 + 13j)
assert (11 + 10j) == laser(input10)

Day 12

In [ ]:
def parse(lines):
    return tuple((integers(line), (0, 0, 0)) for line in lines)


def apply_gravity(moons):
    deltas = ((p1, ((0 if a == b else +1 if a < b else -1)
                    for (a, b) in zip(p1, p2)))
              for ((p1, _), (p2, _)) in permutations(moons, 2))

    incremental = ((sum(x) for x in zip(*(dv for (_, dv) in deltas)))
                   for deltas in chunks(deltas, len(moons) - 1))

    return tuple((p, tuple(sum(x) for x in zip(*(v, dv))))
                 for ((p, v), dv) in zip(moons, incremental))


def apply_velocity(moons):
    return tuple((mapt(sum, zip(p, v)), v) for (p, v) in moons)


def tick(moons):
    return apply_velocity(apply_gravity(moons))


def total_energy(moons):
    return sum(quantify(p, abs) * quantify(v, abs) for (p, v) in moons)


example_1 = '''<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>'''.split('\n')

example_2 = '''<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>'''.split('\n')

assert 179 == total_energy(repeat(tick, 10, parse(example_1)))
assert 1940 == total_energy(repeat(tick, 100, parse(example_2)))

input12 = parse(Input(12, file_template='data/advent2019/input{}.txt'))
assert 13399 == total_energy(repeat(tick, 1000, input12))


In [ ]:
def lcm(a, b):
    return abs(a * b) // gcd(a, b)


def find_lcm(numbers):
    return reduce(lcm, numbers)


def unzip(moons):
    xs = (((x,), (vx,)) for ((x, _, _), (vx, _, _)) in moons)
    ys = (((y,), (vy,)) for ((_, y, _), (_, vy, _)) in moons)
    zs = (((z,), (vz,)) for ((_, _, z), (_, _, vz)) in moons)
    return mapt(tuple, (xs, ys, zs))


def repeats_at(moons):
    start = moons
    for i in range(1, bignum):
        moons = tick(moons)
        if moons == start:
            return i


xs, ys, zs = unzip(parse(example_1))
assert 2772 == find_lcm(map(repeats_at, unzip(parse(example_1))))
assert 4686774924 == find_lcm(map(repeats_at, unzip(parse(example_2))))
# assert 312992287193064 == find_lcm(map(repeats_at, unzip(input12)))